# Anàlisi de dades a gran escala amb spaCy

https://course.spacy.io/chapter2

En aquest capítol, utilitzaràs les teves noves habilitats per extreure informació específica de grans volums de text. Aprendràs com treure el màxim profit de les estructures de dades de spaCy, i com combinar eficaçment enfocaments estadístics i basats en regles per a l'anàlisi de text.

In [ ]:
import spacy

## 1. Estructures de dades (1): Vocab, Lexemes i StringStore

Ara que ja tens experiència real utilitzant els objectes de spaCy, és hora que aprenguis més sobre què passa realment sota el capó de spaCy.

En aquesta lliçó, farem un cop d'ull al vocabulari compartit i com spaCy gestiona les cadenes de text.

### Vocabulari compartit i emmagatzematge de cadenes

SpaCy emmagatzema totes les dades compartides en un vocabulari, el Vocab.

Això inclou paraules, però també els esquemes d'etiquetes per a etiquetes i entitats.

Per estalviar memòria, totes les cadenes es codifiquen com a IDs de hash. Si una paraula apareix més d'una vegada, no cal guardar-la cada vegada.

En canvi, spaCy utilitza una funció de hash per generar un ID i emmagatzemar la cadena només una vegada a l'StringStore.

L'string store està disponible com a nlp.vocab.strings.

És una taula de cerca que funciona en ambdues direccions. Pots cercar una cadena per obtenir el seu hash, o un hash per obtenir la seva cadena.

Internament, spaCy només es comunica amb IDs de hash.

- Vocab: emmagatzema dades compartides entre múltiples documents
- Per estalviar memòria, spaCy codifica totes les cadenes com a hash IDs
- Les cadenes només s'emmagatzemen una vegada a l'StringStore mitjançant nlp.vocab.strings
- String store: taula de cerca en ambdues direccions

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

nlp.vocab.strings.add("coffee")
coffee_hash = nlp.vocab.strings["coffee"]
coffee_string = nlp.vocab.strings[coffee_hash]

print(coffee_hash)
print(coffee_string)

Els IDs de hash no es poden revertir. Si una paraula no és al vocabulari, no hi ha manera d'obtenir la seva cadena. Per això sempre hem de passar el vocabulari compartit.

- Els hashes no es poden revertir – per això hem de proporcionar el vocabulari compartit

In [ ]:
# Això genera un error si no hem vist la cadena abans
string = nlp.vocab.strings[3197928453018144401]

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("I love coffee")
print("hash value:", nlp.vocab.strings["coffee"])
print("string value:", nlp.vocab.strings[3197928453018144401])

El doc també exposa el vocabulari i les cadenes.

In [ ]:
doc = nlp("I love coffee")

print("hash value:", doc.vocab.strings["coffee"])

### Lexemes: entrades al vocabulari

Els lexemes són entrades al vocabulari independents del context.

Obtens un lexeme cercant una cadena o un ID de hash al vocabulari.

Els lexemes exposen atributs, igual que els tokens.

Contenen informació independent del context sobre una paraula.

Text de la paraula: lexeme.text i lexeme.orth (el hash).
    
Atributs lèxics com lexeme.is_alpha.

No conté etiquetes de part del discurs, dependències o etiquetes d'entitat dependents del context

img src="img/data-struct.png" - s'han eliminat les etiquetes per raons de compatibilitat

Un exemple a dalt:

El Doc conté paraules en context – en aquest cas, els tokens "I", "love" i "coffee" amb les seves etiquetes de part del discurs i dependències.

Cada token fa referència a un lexeme, que coneix l'ID de hash de la paraula. Per obtenir la representació en cadena de la paraula, spaCy cerca el hash a l'string store.

## 2. Cadenes a hashes

### Part 1

Cerca la cadena "cat" a nlp.vocab.strings per obtenir el hash.

Cerca el hash per recuperar la cadena.

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("I have a cat")

# Cerca el hash per a la paraula "cat"
cat_hash = nlp.vocab.strings["cat"]
print(cat_hash)

# Cerca cat_hash per obtenir la cadena
cat_string = nlp.vocab.strings[cat_hash]
print(cat_string)

### Part 2

Cerca l'etiqueta de cadena "PERSON" a nlp.vocab.strings per obtenir el hash.

Cerca el hash per recuperar la cadena.

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("David Bowie is a PERSON")

# Cerca el hash per a la cadena "PERSON"
person_hash = nlp.vocab.strings["PERSON"]
print(person_hash)

# Cerca person_hash per obtenir la cadena
person_string = nlp.vocab.strings[person_hash]
print(person_string)

## 3. Vocab, hashes i lexemes

Per què aquest codi genera un error?

In [ ]:
from spacy.lang.de import German
from spacy.lang.en import English

# Crea objectes nlp en anglès i alemany
nlp = English()
nlp_de = German()

# Obté l'ID per a la cadena 'Bowie'
bowie_id = nlp.vocab.strings["Bowie"]
print(bowie_id)

# Cerca l'ID per a 'Bowie' al vocabulari
print(nlp_de.vocab.strings[bowie_id])

La cadena 'Bowie' no és al vocabulari alemany, així que el hash no es pot resoldre a l'string store.

Els hashes no es poden revertir. Per prevenir aquest problema, afegeix la paraula al nou vocabulari processant un text o cercant la cadena, o utilitza el mateix vocabulari per resoldre el hash a una cadena.

## 4. Estructures de dades (2)

Ara que saps tot sobre el vocabulari i l'string store, podem fer un cop d'ull a l'estructura de dades més important: el Doc, i les seves vistes Token i Span.

### L'objecte Doc

El Doc és una de les estructures de dades centrals a spaCy. Es crea automàticament quan processes un text amb l'objecte nlp. Però també pots instanciar la classe manualment.

Després de crear l'objecte nlp, podem importar la classe Doc des de spacy.tokens.

Aquí creem un doc a partir de tres paraules. Els espais són una llista de valors booleans que indiquen si la paraula va seguida d'un espai. Cada token inclou aquesta informació – fins i tot l'últim!

La classe Doc pren tres arguments: el vocabulari compartit, les paraules i els espais.

In [ ]:
# Crea un objecte nlp
import spacy

nlp = spacy.load("en_core_web_sm")

# Importa la classe Doc
from spacy.tokens import Doc

# Les paraules i espais per crear el doc
words = ["Hello", "world", "!"]
spaces = [True, False, False]

# Crea un doc manualment
doc = Doc(nlp.vocab, words=words, spaces=spaces)

### L'objecte Span

Un Span és una porció d'un document que consisteix en un o més tokens. L'Span pren almenys tres arguments: el doc al qual fa referència, i els índexs d'inici i final del span. Recorda que l'índex final és exclusiu!

Per crear un Span manualment, també podem importar la classe des de spacy.tokens. Després podem instanciar-lo amb el doc i els índexs d'inici i final del span, i una etiqueta opcional.

Els doc.ents són escribibles, així que podem afegir entitats manualment sobreescrivint-los amb una llista de spans.

In [ ]:
# Importa les classes Doc i Span
from spacy.tokens import Doc, Span

# Les paraules i espais per crear el doc
words = ["Hello", "world", "!"]
spaces = [True, False, False]

# Crea un doc manualment
doc = Doc(nlp.vocab, words=words, spaces=spaces)

# Crea un span manualment
span = Span(doc, 0, 2)

# Crea un span amb una etiqueta
span_with_label = Span(doc, 0, 2, label="GREETING")

# Afegeix el span a les entitats del doc
doc.ents = [span_with_label]

Alguns consells i trucs abans de començar:

El Doc i l'Span són molt potents i estan optimitzats per al rendiment. Et donen accés a totes les referències i relacions de les paraules i frases.

Si la teva aplicació necessita produir cadenes, assegura't de convertir el doc el més tard possible. Si ho fas massa aviat, perdràs totes les relacions entre els tokens.

Per mantenir la consistència, intenta utilitzar els atributs de token integrats sempre que sigui possible. Per exemple, token.i per a l'índex del token.

A més, no oblidis passar sempre el vocabulari compartit!

<b> Bones pràctiques: </b>

* Doc i Span són molt potents i contenen referències i relacions de paraules i frases

* Converteix els resultats a cadenes el més tard possible
    
* Utilitza atributs de token si estan disponibles – per exemple, token.i per a l'índex del token

* No oblidis passar el vocabulari compartit


## 5. Creant un Doc

Creem alguns objectes Doc des de zero!

### Part 1

Importa el Doc des de spacy.tokens.

Crea un Doc a partir de les paraules i espais. No oblidis passar el vocabulari!

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

# Importa la classe Doc
from spacy.tokens import Doc

# Text desitjat: "spaCy is cool!"
words = ["spaCy", "is", "cool", "!"]
spaces = [True, True, False, False]

# Crea un Doc a partir de les paraules i espais
doc = Doc(nlp.vocab, words=words, spaces=spaces)
print(doc.text)

### Part 2

Importa el Doc des de spacy.tokens.

Crea un Doc a partir de les paraules i espais. No oblidis passar el vocabulari!

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

# Importa la classe Doc
from spacy.tokens import Doc

# Text desitjat: "Go, get started!"
words = ["Go", ",", "get", "started", "!"]
spaces = [False, True, True, False, False]

# Crea un Doc a partir de les paraules i espais
doc = Doc(nlp.vocab, words=words, spaces=spaces)
print(doc.text)

<b> Docstring del DOC: </b>
    
Una seqüència d'objectes Token. Accedeix a frases i entitats anomenades, exporta
anotacions a arrays de numpy, serialitza sense pèrdues a cadenes binàries
comprimides. L'objecte `Doc` conté un array d'estructures `TokenC`. Els
objectes `Token` i `Span` a nivell de Python són vistes d'aquest array, és a dir,
no posseeixen les dades ells mateixos.

EXEMPLE: Construcció 1
    >>> doc = nlp(u'Some text')

    Construcció 2
    >>> from spacy.tokens import Doc
    >>> doc = Doc(nlp.vocab, words=[u'hello', u'world', u'!'],
                  spaces=[True, False, False])
Docstring d'inicialització:
Crea un objecte Doc.

vocab (Vocab): Un objecte de vocabulari, que ha de coincidir amb qualsevol model que
    vulguis utilitzar (per exemple, tokenitzador, analitzador sintàctic, reconeixedor d'entitats).
    
words (list o None): Una llista de cadenes unicode per afegir al document
    com a paraules. Si és `None`, per defecte és una llista buida.
    
spaces (list o None): Una llista de valors booleans, de la mateixa longitud que
    words. True significa que la paraula va seguida d'un espai, False significa
    que no. Si és `None`, per defecte és `[True]*len(words)`
    
user_data (dict o None): Dades opcionals del programa, atributs no estructurats
    (sense semàntica) adjunts al Doc.

### Part 3

Importa el Doc des de spacy.tokens.

Crea un Doc a partir de les paraules i espais. No oblidis passar el vocabulari!

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")

# Importa la classe Doc
from spacy.tokens import Doc

# Text desitjat: "Oh, really?!"
words = ["Oh", ",", "really", "?", "!"]
spaces = [False, True, False, False, False]

# Crea un Doc a partir de les paraules i espais
doc = Doc(nlp.vocab, words=words, spaces=spaces)
print(doc.text)

## 6. Docs, spans i entitats des de zero

En aquest exercici, crearàs els objectes Doc i Span manualment, i actualitzaràs les entitats anomenades – igual que ho fa spaCy entre bastidors. Es crearà un objecte nlp compartit sense cap pipeline.

- Importa les classes Doc i Span des de spacy.tokens.
- Utilitza la classe Doc directament per crear un doc a partir de les paraules i espais.
- Crea un Span per a "David Bowie" del doc i assigna-li l'etiqueta "PERSON".
- Sobreescriu doc.ents amb una llista d'una entitat, l'span "David Bowie".

In [ ]:
from spacy.lang.ca import Catalan

nlp = Catalan()

# Importa les classes Doc i Span
from spacy.tokens import Doc, Span

words = ["M'", "encanta", "en", "David", "Bowie"]
spaces = [False, True, True, True, False]

# Crea un doc a partir de les paraules i espais
doc = Doc(nlp.vocab, words=words, spaces=spaces)
print(doc.text)

# Crea un span per a "David Bowie" del doc i assigna-li l'etiqueta "PERSON"
span = Span(doc, 3, 5, label="PERSON")
print(span.text, span.label_)

# Afegeix el span a les entitats del doc
doc.ents = [span]

# Imprimeix el text i les etiquetes de les entitats
print([(ent.text, ent.label_) for ent in doc.ents])

## 7. Estructures de dades i bones pràctiques

El codi d'aquest exemple intenta analitzar un text i recollir tots els noms propis que van seguits d'un verb.

### Part 1

Per què el codi és dolent?

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("Berlin is a nice city")

# Obté tots els tokens i etiquetes de part del discurs
token_texts = [token.text for token in doc]
pos_tags = [token.pos_ for token in doc]

for index, pos in enumerate(pos_tags):
    # Comprova si el token actual és un nom propi
    if pos == "PROPN":
        # Comprova si el següent token és un verb
        if pos_tags[index + 1] == "VERB":
            result = token_texts[index]
            print("Trobat nom propi abans d'un verb:", result)

In [ ]:
print(token_texts)
print(pos_tags)

In [ ]:
print(spacy.explain("PROPN"))
print(spacy.explain("VERB"))
print(spacy.explain("DET"))
print(spacy.explain("ADJ"))
print(spacy.explain("NOUN"))

Resposta: 

Només utilitza llistes de cadenes en lloc d'atributs natius de token. Això sovint és menys eficient, i no pot expressar relacions complexes.

Converteix sempre els resultats a cadenes el més tard possible, i intenta utilitzar atributs natius de token per mantenir la consistència.

Part 2
Reescriu el codi per utilitzar els atributs natius de token en lloc de llistes de cadenes.

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("Berlin is a nice city")

for token in doc:
    # Comprova si el token actual és un nom propi
    if token.pos_ == "PROPN":
        # Comprova si el següent token és un verb
        if doc[token.i + 1].pos_ == "VERB":
            print("Trobat nom propi abans d'un verb:", token.text)

## 8. Vectors de paraules i similituds semàntiques

En aquesta lliçó, aprendràs a utilitzar spaCy per predir com de semblants són documents, spans o tokens entre si.

També aprendràs sobre com utilitzar vectors de paraules i com treure'n profit a la teva aplicació de PLN.

spaCy pot comparar dos objectes i predir com de semblants són – per exemple, documents, spans o tokens individuals.

Els objectes Doc, Token i Span tenen un mètode dot similarity que pren un altre objecte i retorna un nombre de coma flotant entre 0 i 1, indicant com de semblants són.

Una cosa molt important: Per utilitzar la similitud, necessites un model de spaCy més gran que inclogui vectors de paraules.

<b>Per exemple, el model anglès mitjà o gran – però no el petit. Així que si vols utilitzar vectors, sempre utilitza un model que acabi en "md" o "lg". Pots trobar més detalls a la documentació de models.</b>

- spaCy pot comparar dos objectes i predir la similitud
- Doc.similarity(), Span.similarity() i Token.similarity()
- Prenen un altre objecte i retornen un valor de similitud (entre 0 i 1)
- Important: necessita un model que tingui vectors de paraules inclosos, per exemple en_core_web_md (mitjà) o en_core_web_lg (gran)
- No en_core_web_sm (petit)

In [ ]:
import spacy

# Carrega un model més gran amb vectors
nlp = spacy.load("en_core_web_md")

# Compara dos documents
doc1 = nlp("I like fast food")
doc2 = nlp("I like pizza")
print(doc1.similarity(doc2))

In [ ]:
# Compara dos tokens
doc = nlp("I like pizza and pasta")
token1 = doc[2]
token2 = doc[4]
print(token1.similarity(token2))

In [ ]:
# Compara un document amb un token
doc = nlp("I like pizza")
token = nlp("soap")[0]

print(doc.similarity(token))

In [ ]:
# Compara un span amb un document
span = nlp("I like pizza and pasta")[2:5]
doc = nlp("McDonalds sells burgers")

print(span.similarity(doc))

Però com ho fa spaCy internament?

La similitud es determina utilitzant vectors de paraules, representacions multidimensionals dels significats de les paraules.

Potser has sentit parlar de Word2Vec, que és un algorisme que s'utilitza sovint per entrenar vectors de paraules a partir de text en brut.

Els vectors es poden afegir als models estadístics de spaCy.

Per defecte, la similitud retornada per spaCy és la similitud del cosinus entre dos vectors – però això es pot ajustar si és necessari.

Els vectors per a objectes que consisteixen en diversos tokens, com el Doc i l'Span, per defecte són la mitjana dels seus vectors de token.

Per això normalment obtens més valor de frases més curtes amb menys paraules irrellevants.

Com prediu spaCy la similitud?
- La similitud es determina utilitzant vectors de paraules
- Representacions multidimensionals del significat de les paraules
- Generats utilitzant un algorisme com Word2Vec i molts textos
- Es poden afegir als models estadístics de spaCy
- Per defecte: similitud del cosinus, però es pot ajustar
- Els vectors de Doc i Span per defecte són la mitjana dels vectors dels tokens
- Les frases curtes són millors que les llargues amb moltes paraules irrellevants

In [ ]:
doc = nlp("I have a banana")
# Accedeix al vector mitjançant l'atribut token.vector
print(doc[3].vector)

### La similitud depèn del context de l'aplicació

Útil per a moltes aplicacions: sistemes de recomanació, marcar duplicats, etc.

No hi ha cap definició objectiva de "similitud"

Depèn del context i del que l'aplicació necessiti fer.

Aquí tens un exemple: els vectors de paraules per defecte de spaCy assignen una puntuació de similitud molt alta a "I like cats" i "I hate cats". Això té sentit, perquè tots dos textos expressen sentiment sobre gats. Però en un context d'aplicació diferent, potser voldries considerar les frases com a molt diferents, perquè parlen de sentiments oposats.

In [ ]:
doc1 = nlp("I like cats")
doc2 = nlp("I hate cats")

print(doc1.similarity(doc2))

- Útil per a moltes aplicacions: sistemes de recomanació, marcar duplicats, etc.
- No hi ha cap definició objectiva de "similitud"
- Depèn del context i del que l'aplicació necessiti fer

## 9. Inspeccionant vectors de paraules

En aquest exercici, utilitzaràs un model anglès més gran, que inclou al voltant de 20.000 vectors de paraules. Com que els vectors triguen una mica més a carregar, estem utilitzant una versió lleugerament comprimida del que pots descarregar amb spaCy. El model ja està preinstal·lat.

- Carrega el model mitjà 'en_core_web_md' amb vectors de paraules.
- Imprimeix el vector per a "bananas" utilitzant l'atribut token.vector.


In [ ]:
import spacy

# Carrega el model mitjà en_core_web_md
nlp = spacy.load("en_core_web_md")

# Processa un text
doc = nlp("Two bananas in pyjamas")

# Obté el vector per al token "bananas"
bananas_vector = doc[1].vector
print(bananas_vector)

## 10. Comparant similituds

En aquest exercici, utilitzaràs el mètode similarity de spaCy per comparar objectes Doc, Token i Span i obtenir puntuacions de similitud.

### Part 1

- Utilitza el mètode doc.similarity per comparar doc1 amb doc2 i imprimeix el resultat.

In [ ]:
import spacy

nlp = spacy.load("en_core_web_md")

doc1 = nlp("It's a warm summer day")
doc2 = nlp("It's sunny outside")

# Obté la similitud de doc1 i doc2
similarity = doc1.similarity(doc2)
print(similarity)

### Part 2

- Utilitza el mètode token.similarity per comparar token1 amb token2 i imprimeix el resultat.

In [ ]:
import spacy

nlp = spacy.load("en_core_web_md")

doc = nlp("TV and books")
token1, token2 = doc[0], doc[2]

# Obté la similitud del token "TV" i "books"
similarity = token1.similarity(token2)
print(similarity)

### Part 3

Crea spans per a "great restaurant"/"really nice bar".

Utilitza span.similarity per comparar-los i imprimeix el resultat.

In [ ]:
import spacy

nlp = spacy.load("en_core_web_md")

doc = nlp("This was a great restaurant. Afterwards, we went to a really nice bar.")

# Crea spans per a "great restaurant" i "really nice bar"
span1 = doc[3:5]
span2 = doc[12:15]

# Obté la similitud dels spans
similarity = span1.similarity(span2)
print(similarity)

In [ ]:
doc = nlp("M'agrada una tassa de cafè calent però odio un pa fred amb mantega.")

span1 = doc[0:7]
span2 = doc[8:15]

# Obté la similitud dels spans
similarity = span1.similarity(span2)
print(similarity)

## 11. Combinant models i regles

Combinar models estadístics amb sistemes basats en regles és un dels trucs més potents que hauries de tenir a la teva caixa d'eines de PLN.

En aquesta lliçó, farem un cop d'ull a com fer-ho amb spaCy


### Prediccions estadístiques vs. regles 

Els models estadístics són útils si la teva aplicació necessita ser capaç de generalitzar basant-se en uns pocs exemples.

Per exemple, detectar noms de productes o persones normalment es beneficia d'un model estadístic. En lloc de proporcionar una llista de tots els noms de persones de la història, la teva aplicació podrà predir si un span de tokens és un nom de persona. De manera similar, pots predir etiquetes de dependència per trobar relacions subjecte/objecte.

Per fer això, utilitzaries el reconeixedor d'entitats, l'analitzador de dependències o l'etiquetador de parts del discurs de spaCy.

Els enfocaments basats en regles, d'altra banda, són útils si hi ha un nombre més o menys finit d'instàncies que vols trobar. Per exemple, tots els països o ciutats del món, noms de medicaments o fins i tot races de gossos.

A spaCy, pots aconseguir això amb regles de tokenització personalitzades, així com el matcher i phrase matcher.

img src="img/vs.png"

### RESUM: Coincidència basada en regles

Al capítol anterior, vas aprendre a utilitzar el matcher basat en regles de spaCy per trobar patrons complexos als teus textos. Aquí tens un resum ràpid.

El matcher s'inicialitza amb el vocabulari compartit – normalment nlp.vocab.

Els patrons són llistes de diccionaris, i cada diccionari descriu un token i els seus atributs. Els patrons es poden afegir al matcher utilitzant el mètode matcher.add.

Els operadors et permeten especificar quantes vegades coincidir un token. Per exemple, "+" coincidirà una o més vegades.

Cridar el matcher sobre un objecte doc retornarà una llista de les coincidències. Cada coincidència és una tupla que consisteix en un ID, i l'índex de token d'inici i final al document.

In [ ]:
# Inicialitza amb el vocabulari compartit
from spacy.matcher import Matcher

matcher = Matcher(nlp.vocab)

# Els patrons són llistes de diccionaris que descriuen els tokens
pattern = [{"LEMMA": "love", "POS": "VERB"}, {"LOWER": "cats"}]
matcher.add("LOVE_CATS", None, pattern)

# Els operadors poden especificar quantes vegades s'ha de coincidir un token
pattern = [{"TEXT": "very", "OP": "+"}, {"TEXT": "happy"}]

# Cridar matcher sobre doc retorna una llista de tuples (match_id, start, end)
doc = nlp("I love cats and I'm very very happy")
matches = matcher(doc)

In [ ]:
matches

RETORNA (list): Una llista de tuples `(key, start, end)`,
    descrivint les coincidències. Una tupla de coincidència descriu un span
    `doc[start:end]`. El `label_id` i `key` són tots dos enters.

### Afegint prediccions estadístiques 

Aquí tens un exemple d'una regla de matcher per a "golden retriever".

Si iterem sobre les coincidències retornades pel matcher, podem obtenir l'ID de coincidència i l'índex d'inici i final del span coincident. Després podem descobrir més sobre ell. Els objectes Span ens donen accés al document original i a tots els altres atributs de token i característiques lingüístiques predites pel model.

Per exemple, podem obtenir el token arrel del span. Si el span consisteix en més d'un token, aquest serà el token que decideix la categoria de la frase. Per exemple, l'arrel de "Golden Retriever" és "Retriever". També podem trobar el token cap de l'arrel. Aquest és el "pare" sintàctic que governa la frase – en aquest cas, el verb "have".

Finalment, podem mirar el token anterior i els seus atributs. En aquest cas, és un determinant, l'article "a".

In [ ]:
matcher = Matcher(nlp.vocab)
matcher.add("DOG", None, [{"LOWER": "golden"}, {"LOWER": "retriever"}])
doc = nlp("I have a Golden Retriever")

for match_id, start, end in matcher(doc):
    span = doc[start:end]
    print("Span coincident:", span.text)
    # Obté el token arrel del span i el token cap de l'arrel
    print("Token arrel:", span.root.text)
    print("Token cap de l'arrel:", span.root.head.text)
    # Obté el token anterior i la seva etiqueta POS
    print("Token anterior:", doc[start - 1].text, doc[start - 1].pos_)

### Coincidència eficient de frases (1) 

El phrase matcher és una altra eina útil per trobar seqüències de paraules a les teves dades.

Fa una cerca de paraules clau al document, però en lloc de trobar només cadenes, et dóna accés directe als tokens en context.

Pren objectes Doc com a patrons.

També és molt ràpid.

Això el fa molt útil per coincidir grans diccionaris i llistes de paraules en grans volums de text.

- PhraseMatcher com expressions regulars o cerca de paraules clau – però amb accés als tokens!
- Pren objectes Doc com a patrons
- Més eficient i ràpid que el Matcher
- Ideal per coincidir grans llistes de paraules

Aquí tens un exemple.

El phrase matcher es pot importar des de spacy.matcher i segueix la mateixa API que el matcher normal.

En lloc d'una llista de diccionaris, passem un objecte Doc com a patró.

Després podem iterar sobre les coincidències al text, que ens dóna l'ID de coincidència, i l'inici i final de la coincidència. Això ens permet crear un objecte Span per als tokens coincidents "Golden Retriever" per analitzar-lo en context.

In [ ]:
from spacy.matcher import PhraseMatcher

matcher = PhraseMatcher(nlp.vocab)

pattern = nlp("Golden Retriever")
matcher.add("DOG", None, pattern)
doc = nlp("I have a Golden Retriever")

# Itera sobre les coincidències
for match_id, start, end in matcher(doc):
    # Obté el span coincident
    span = doc[start:end]
    print("Span coincident:", span.text)

## 12. Depurant patrons (1)

Per què aquest patró no coincideix amb els tokens "Silicon Valley" al doc?

In [ ]:
pattern = [{"LOWER": "silicon"}, {"TEXT": " "}, {"LOWER": "valley"}]

doc = nlp("Can Silicon Valley workers.")

Els tokens no inclouen espais. L'atribut de text d'un token retorna el text del token sense espais. No has de buscar espais!

## 13. Depurant patrons (2)

Tots dos patrons d'aquest exercici contenen errors i no coincidiran com s'esperava. Pots arreglar-los? Si et quedes encallat, intenta imprimir els tokens al doc per veure com es divideix el text i ajusta el patró perquè cada diccionari representi un token.

- Edita pattern1 perquè coincideixi correctament amb tots els casos de "Amazon" més un títol en majúscula i nom propi.
- Edita pattern2 perquè coincideixi correctament amb totes les formes de "ad-free", més un substantiu.

In [ ]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")
doc = nlp(
    "Twitch Prime, the perks program for Amazon Prime members offering free "
    "loot, games and other benefits, is ditching one of its best features: "
    "ad-free viewing. According to an email sent out to Amazon Prime members "
    "today, ad-free viewing will no longer be included as a part of Twitch "
    "Prime for new members, beginning on September 14. However, members with "
    "existing annual subscriptions will be able to continue to enjoy ad-free "
    "viewing until their subscription comes up for renewal. Those with "
    "monthly subscriptions will have access to ad-free viewing until October 15."
)

# Crea els patrons de coincidència
pattern1 = [{"LOWER": "Amazon"}, {"IS_TITLE": True, "POS": "PROPN"}]
pattern2 = [{"LOWER": "ad-free"}, {"POS": "NOUN"}]

# Inicialitza el Matcher i afegeix els patrons
matcher = Matcher(nlp.vocab)
matcher.add("PATTERN1", None, pattern1)
matcher.add("PATTERN2", None, pattern2)

# Itera sobre les coincidències
for match_id, start, end in matcher(doc):
    # Imprimeix el nom de cadena del patró i el text del span coincident
    print(doc.vocab.strings[match_id], doc[start:end].text)

In [ ]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")
doc = nlp(
    "Twitch Prime, the perks program for Amazon Prime members offering free "
    "loot, games and other benefits, is ditching one of its best features: "
    "ad-free viewing. According to an email sent out to Amazon Prime members "
    "today, ad-free viewing will no longer be included as a part of Twitch "
    "Prime for new members, beginning on September 14. However, members with "
    "existing annual subscriptions will be able to continue to enjoy ad-free "
    "viewing until their subscription comes up for renewal. Those with "
    "monthly subscriptions will have access to ad-free viewing until October 15."
)

# Crea els patrons de coincidència
pattern1 = [{"LOWER": "amazon"}, {"IS_TITLE": True, "POS": "PROPN"}]
pattern2 = [{"LOWER": "ad"}, {"TEXT": "-"}, {"LOWER": "free"}, {"POS": "NOUN"}]

# Inicialitza el Matcher i afegeix els patrons
matcher = Matcher(nlp.vocab)
matcher.add("PATTERN1", None, pattern1)
matcher.add("PATTERN2", None, pattern2)

# Itera sobre les coincidències
for match_id, start, end in matcher(doc):
    # Imprimeix el nom de cadena del patró i el text del span coincident
    print(doc.vocab.strings[match_id], doc[start:end].text)

## 14. Coincidència eficient de frases

De vegades és més eficient coincidir cadenes exactes en lloc d'escriure patrons que descriguin els tokens individuals. Això és especialment cert per a categories finites de coses – com tots els països del món. Ja tenim una llista de països, així que utilitzem-la com a base del nostre script d'extracció d'informació. Una llista de noms de cadena està disponible com a variable COUNTRIES.

- Importa el PhraseMatcher i inicialitza'l amb el vocabulari compartit com a variable matcher.
- Afegeix els patrons de frase i crida el matcher sobre el doc.

In [ ]:
import json

from spacy.lang.en import English

with open("exercises/countries.json") as f:
    COUNTRIES = json.loads(f.read())

nlp = English()
doc = nlp("Czech Republic may help Slovakia protect its airspace")

# Import the PhraseMatcher and initialize it
from spacy.matcher import PhraseMatcher

matcher = PhraseMatcher(nlp.vocab)

# Create pattern Doc objects and add them to the matcher
# This is the faster version of: [nlp(country) for country in COUNTRIES]
patterns = list(nlp.pipe(COUNTRIES))
matcher.add("COUNTRY", None, *patterns)

# Call the matcher on the test document and print the result
matches = matcher(doc)
print([doc[start:end] for match_id, start, end in matches])

### Extraient països i relacions ----

A l'exercici anterior, vas escriure un script utilitzant el PhraseMatcher de spaCy per trobar noms de països al text. Utilitzem aquest matcher de països en un text més llarg, analitzem la sintaxi i actualitzem les entitats del document amb els països coincidents.

- Itera sobre les coincidències i crea un Span amb l'etiqueta "GPE" (entitat geopolítica).
- Sobreescriu les entitats a doc.ents i afegeix el span coincident.
- Obté el token cap de l'arrel del span coincident.
- Imprimeix el text del token cap i el span.

In [ ]:
import json

from spacy.lang.en import English
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span

with open("exercises/countries.json") as f:
    COUNTRIES = json.loads(f.read())
with open("exercises/country_text.txt") as f:
    TEXT = f.read()

nlp = English()
matcher = PhraseMatcher(nlp.vocab)
patterns = list(nlp.pipe(COUNTRIES))
matcher.add("COUNTRY", None, *patterns)

# Crea un doc i troba coincidències
doc = nlp(TEXT)

# Itera sobre les coincidències
for match_id, start, end in matcher(doc):
    # Crea un Span amb l'etiqueta "GPE"
    span = Span(doc, start, end, label="GPE")

    # Sobreescriu doc.ents i afegeix el span
    doc.ents = list(doc.ents) + [span]

    # Obté el token cap de l'arrel del span
    span_root_head = span.root.head
    # Imprimeix el text del token cap de l'arrel del span i el text del span
    print(span_root_head.text, "-->", span.text)

# Imprimeix les entitats del document
print([(ent.text, ent.label_) for ent in doc.ents if ent.label_ == "GPE"])